Problem Statement. <br/>

Design a data structure that follows the constraints of a Least Recently Used (LRU) cache. <br/>

Implement the LRUCache class: <br/>

    LRUCache(int capacity) Initialize the LRU cache with positive size capacity. <br/>
    int get(int key) Return the value of the key if the key exists, otherwise return -1. <br/>
    void put(int key, int value) Update the value of the key if the key exists. Otherwise, add the key-value pair to the cache. If the number of keys exceeds the capacity from this operation, evict the least recently used key.


# Hash Map - O(1) Get, O(N) Put, O(C) space where C is the capacity

In [17]:
from datetime import datetime

class LRUCache:

    def __init__(self, capacity: int):
        self.capacity = capacity
        self.cache = {}
        

    def get(self, key: int) -> int:
        
        if key in self.cache:
            dt = datetime.now()
            seq = int(dt.strftime("%Y%m%d%H%M%S%f"))
            self.cache[key][0] = seq
            return self.cache[key][1]
        
        return -1
        

    def put(self, key: int, value: int) -> None:
        dt = datetime.now()
        seq = int(dt.strftime("%Y%m%d%H%M%S%f"))
        if key not in self.cache and len(self.cache) == self.capacity:
            last_accessed = min(self.cache.items(), key=lambda x: x[1][0])
            self.cache.pop(last_accessed[0])
        self.cache[key] = [seq, value]

# Hash Map + Doubly Linked Node - O(1) Get, O(1) Put, O(C) space where C is the capacity

In [21]:
class DLinkedNode(): 
    def __init__(self):
        self.key = 0
        self.value = 0
        self.prev = None
        self.next = None

class LRUCache(OrderedDict):
    
    def _add_node(self, node: DLinkedNode):
        """
        Always add the new node right after head.
        """
        node.prev = self.head
        node.next = self.head.next

        self.head.next.prev = node
        self.head.next = node

    def _remove_node(self, node: DLinkedNode):
        """
        Remove an existing node from the linked list.
        """
        prev = node.prev
        new = node.next

        prev.next = new
        new.prev = prev

    def _move_to_head(self, node: DLinkedNode):
        """
        Move certain node in between to the head.
        """
        self._remove_node(node)
        self._add_node(node)

    def _pop_tail(self) -> DLinkedNode:
        """
        Pop the current tail.
        """
        res = self.tail.prev
        self._remove_node(res)
        return res

    def __init__(self, capacity: int):
        self.cache = {}
        self.size = 0
        self.capacity = capacity
        self.head, self.tail = DLinkedNode(), DLinkedNode()

        self.head.next = self.tail
        self.tail.prev = self.head
        

    def get(self, key: int) -> int:
        
        node = self.cache.get(key, None)
        if not node:
            return -1

        # move the accessed node to the head;
        self._move_to_head(node)

        return node.value
        

    def put(self, key: int, value: int) -> None:
        node = self.cache.get(key)

        if not node: 
            newNode = DLinkedNode()
            newNode.key = key
            newNode.value = value

            self.cache[key] = newNode
            self._add_node(newNode)

            self.size += 1

            if self.size > self.capacity:
                # pop the tail
                tail = self._pop_tail()
                del self.cache[tail.key]
                self.size -= 1
        else:
            # update the value.
            node.value = value
            self._move_to_head(node)

# Ordered Dict - O(1) Get, O(1) Put, O(C) space where C is the capacity

In [19]:
from collections import OrderedDict

class LRUCache(OrderedDict):

    def __init__(self, capacity: int):
        self.capacity = capacity
        

    def get(self, key: int) -> int:
        
        if key not in self:
            return - 1
        
        self.move_to_end(key)
        return self[key]
        

    def put(self, key: int, value: int) -> None:
        if key in self:
            self.move_to_end(key)
        self[key] = value
        if len(self) > self.capacity:
            self.popitem(last = False)

In [22]:
lRUCache = LRUCache(2);
lRUCache.put(1, 1); # cache is {1=1}
lRUCache.put(2, 2); # cache is {1=1, 2=2}
print(lRUCache.get(1));    # return 1
lRUCache.put(3, 3); # LRU key was 2, evicts key 2, cache is {1=1, 3=3}
print(lRUCache.get(2));    # returns -1 (not found)
lRUCache.put(4, 4); # LRU key was 1, evicts key 1, cache is {4=4, 3=3}
print(lRUCache.get(1));    # return -1 (not found)
print(lRUCache.get(3));    # return 3
print(lRUCache.get(4));    # return 4

1
-1
-1
3
4
